In [92]:
import pandas as pd 

df = pd.read_csv('hotel_booking.csv')

df.head(2)

# df.info()
# null_counts = df.isnull().sum()
# null_counts

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157


## 1 - convert the all object columns to number for random imputer

In [93]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cat_cols = df.select_dtypes(include=['object']).columns
print(cat_cols)


for col in cat_cols:
    df[col] = le.fit_transform(df[col])

df[cat_cols] = df[cat_cols].apply(lambda col: le.fit_transform(col))
print(df.isna().sum())



Index(['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'customer_type', 'reservation_status',
       'reservation_status_date', 'name', 'email', 'phone-number',
       'credit_card'],
      dtype='object')
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                                0
market_segment                         0
distribution_channel                   0
is_repeated_guest   

In [94]:
columns_with_nan = df.columns[df.isna().any()].tolist()

for column in columns_with_nan:
    random_values = df[column].dropna().sample(df[column].isna().sum(), replace=True)
    
    df[column].loc[df[column].isna()] = random_values.values

print(df.isna().sum())


hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

C:\Users\ehsan\AppData\Local\Temp\ipykernel_13660\2003169365.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].loc[df[column].isna()] = random_values.values


## step 2 : select data for train and target

In [95]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np 
df = df.iloc[ : , : ].values

x = np.delete(df, [1,-1,-2,-3,-4,-12,-13],axis=1)
y = df[ : ,1 ]



## step 3 : Data normalization (we put all the data in the same range)

In [96]:
from sklearn.preprocessing import MaxAbsScaler 
scaler = MaxAbsScaler()
x = scaler.fit_transform(x)

print("Result :")
t = x
print(t[0])


Result :
[1.         0.46404342 0.99900843 0.45454545 0.50943396 0.03225806
 0.         0.         0.03636364 0.         0.         0.
 0.76271186 0.42857143 0.25       0.         0.         0.
 0.22222222 0.18181818 0.14285714 0.         0.         0.66666667
 0.         0.         0.         0.5        0.13081081]


## KNN ALGORITHM

In [97]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
param_grid = {
    "n_neighbors": [3, 5, 7],
    "weights": ["uniform", "distance"],
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_knn = KNeighborsClassifier(**best_params)
best_knn.fit(X_train, y_train)
accuracy = best_knn.score(X_test, y_test)

print("دقت:", accuracy)

دقت: 0.9982410587151352


## SVM ALGORITHM

In [98]:
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
param_grid = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf", "poly"],
}

svm = SVC()

grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_svm = SVC(**best_params)
best_svm.fit(X_train, y_train)

accuracy = best_svm.score(X_test, y_test)
print("دقت:", accuracy)

دقت: 1.0


##  Random Forest  ALGORITHM

In [99]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, None],
}

random_forest = RandomForestClassifier()
grid_search = GridSearchCV(random_forest, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_random_forest = RandomForestClassifier(**best_params)
best_random_forest.fit(X_train, y_train)
accuracy = best_random_forest.score(X_test, y_test)

print("دقت:", accuracy)


دقت: 1.0


## Decision Tree ALGORITHM

In [100]:
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 5, 10],
}

decision_tree = DecisionTreeClassifier()
grid_search = GridSearchCV(decision_tree, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_decision_tree = DecisionTreeClassifier(**best_params)
best_decision_tree.fit(X_train, y_train)
accuracy = best_decision_tree.score(X_test, y_test)

print("Accuracy:", accuracy)

دقت: 1.0
